Generic Instructions for Creating  New Reference Files
=======================================

Tutorial Approach
---------------------

We recommend that you review the tutorial for creating Flat Field Reference files 
before doing this one.

Rather than create a tutorial for each specific kind of reference file, this
tutorial will show how to use either the schema for that reference file as a 
guide, or a simplified representation of the schema (the latter should be much
easier, and such information will be added to this tutorial as new reference 
file types are added.

This tutorial will use a few examples of how to do this,
and we expect that it will illustrate how to do the same for any reference file type.
More than one example is necessary since no particular one addresses all the 
various aspects one may encounter.

First, common elements to all reference files. All reference files have a meta
attribute, so one may do the following for all cases

In [ ]:
import roman_datamodels.datamodels as rdd
import roman_datamodels.stnode as rds
import numpy as np
import asdf
from astropy.time import Time
# create meta dictionary and populate with common elements
meta = {'telescope': 'ROMAN', 'reftype': '<the expected value for this refernece file>'}
# Create the information about instrument configuration for this flat field.
# The first item is always the same, but the others depend on the particular flat field
instrument = {'name': 'WFI', 'detector': 'WFI01', 'optical_element': 'F129'}
meta['instrument'] = instrument
# All the rest of these must be provided by the person who has constructed the flat field
meta['pedigree'] = """whatever text you wish to describe the pedigree
    and it may go on and on and on and on and on and on
    and on and on and on and on and on
    and on and on, to your heart's content"""
meta['description'] = "whatever description you wish to provide"
meta['author'] = 'Wally Wombat'
meta['useafter'] = Time('2022-01-01T11:11:11.111') # the good old useafter date
meta['origin'] = 'STScI'
meta['telescope'] = 'ROMAN'

At this point create the object corresponding to the reference file type. For flat fields it would be FlatRef. Generally speaking the Class name is derived from the schema name. For examples (including the required value of reftype:

| Reference Name | Class Name | reftype value |
| :- | :- | :- |
| dark | DarkRef | DARK |
| flat | FlatRef | FLAT |
| gain | GainRef | GAIN |
| mask | MaskRef | MASK |
| readnoise | ReadnoiseRef | READNOISE |


Handling dq_def
-------------------

Many, if not most reference files currently expect a definition of which
data quality items they use in their own DQ array. This is handled by the
`dq_def` item, which essentially identifies which bit plane in the reference
file DQ array is associated with the standard data file DQ mnemonic. (In 
reading in the reference file DQ array, the calibration step maps the DQ
array bit planes into a full blown DQ array used by the calibration data.)
This is done to reduce the number of bits needed by the reference file DQ 
array (e.g., the byte size of each pixel in the DQ array).

The following illustrates how to create the `dq_def` item. Since the details
are not documented anywhere except by the `dq_def` item itself, this tutorial
cannot tell one which DQ mnemonics must be defined. This is up to each
instrument team. The example is a made up one and does not correspond to the
one that must be defined for any particular reference file type.

Complicating this example is that Roman has not yet come up with a definitive
list of mnemonics. This tutorial will use some basic ones that very likely 
will be present.

The dq_def item is a table consisting of four columns:

- Bit number (starting at 0)
- Bit value (e.g., 1, 2, 4, 8, etc)
- Bit name (i.e., the standard mnemonic), maximum of 40 characters
- Bit description, maximum of 80 characters

We will construct an example of such a table using numpy using the mnemonics of:

- DO_NOT_USE
- DEAD
- HOT

In [ ]:
dq_def = np.array([
    (0, 1, 'DO_NOT_USE', 'Bad pixel. Do not use'),
    (1, 2, 'DEAD', 'Dead pixel'),
    (2, 4, 'HOT', 'Hot pixel')
    ], dtype=[('BIT', np.int32), ('VALUE', np.uint32), ('NAME', 'S40'), ('DESCRIPTION', 'S80')])

In [ ]:
dq_def

Handling reference file specific attributes. Check this notebook for the current set of expected attributes for each reference file type: [Reference Files Specifics](./ReferenceFile_Required_Attributes.ipynb). To take as an example making a mask reference file. This requires only one more attribute, the actual DQ array. In this case it will be all 0's.

In [ ]:
meta['reftype'] = 'MASK'
mask = rds.MaskRef()
mask['meta'] = meta
mask['dq_def'] = dq_def
mask['dq'] = np.zeros((100,100), dtype=np.uint16) # whatever shape is appropriate
af = asdf.AsdfFile()
af.tree = {'roman': mask}
af.write_to('my_maskref.asdf')

Check the new file

In [ ]:
mask2 = rdd.open('my_maskref.asdf')

In [ ]:
type(mask2)

In [ ]:
mask2.meta

In [ ]:
mask2.dq_def

In [ ]:
mask2.dq.shape